<a href="https://colab.research.google.com/github/Shinoj28/DL/blob/main/DLP4_Using_deep_feed_forward_network_with_two_hidden_layer_for_performing_classification_and_predicting_the_probability_of_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from keras.layers import Dense, Input, concatenate, Dropout
from keras.models import Model
from tensorflow.keras import optimizers
optimizers.RMSprop
optimizers.Adam

keras.src.optimizers.adam.Adam

In [2]:
dataset = load_wine()

In [3]:
#number of sub-networks
ensemble_num = 10

In [4]:
#80% size of original (training) dataset
bootstrap_size = 0.8

In [5]:
#80% size for training, 20% for test
training_size = 0.8

In [6]:
#number of neurons in hidden layer
num_hidden_neurosn = 10
#percentage of weights dropped out before softmax ouput (this prevents overfitting)
dropout = 0.25

In [7]:
# number of epochs(complete training episodes over the training set) to run
epochs = 100

In [8]:
# mini batch size for better convergence
batch = 10

In [9]:
# get the holdout training and test set
temp = []
scaler = MinMaxScaler()

In [10]:
# one hot encode the target classes
one_hot = OneHotEncoder()
dataset['data'] = scaler.fit_transform(dataset['data'])
dataset['target'] = one_hot.fit_transform(np.reshape(dataset['target'], (-1, 1))).toarray()
for i in range(len(dataset.data)):
    temp.append([dataset['data'][i], np.array(dataset['target'][i])])

In [11]:
# shuffle the row of data and targets
temp = np.array(temp, dtype=object)
np.random.shuffle(temp)
#holdout training and test stop index
stop = int(training_size*len(dataset.data))

In [12]:
train_X = np.array([x for x in temp[:stop,  0]])
train_Y = np.array([x for x in temp[:stop,  1]])
test_X = np.array([x for x in temp[stop:,  0]])
test_Y = np.array([x for x in temp[stop:,  1]])

In [13]:
# now build the ensemble neural network
#first, let's build the individual sub-networks, each
# as a keras functional model.

num_hidden_neurons = 64

sub_net_outputs = []
sub_net_inputs = []
for i in range(ensemble_num):
  #two hidden layers to keep it simple
  # specify input shape to the shape of the training set
  net_input = Input(shape=(train_X.shape[1],))
  sub_net_inputs.append(net_input)
  Y = Dense(num_hidden_neurons)(net_input)
  Y = Dense(num_hidden_neurons)(Y)
  Y = Dropout(dropout)(Y)

  # sub_nets contains the output tensors
  sub_net_outputs.append(Y)


In [14]:
# now concatenate the ouput tensors
Y = concatenate(sub_net_outputs)

In [15]:
# final softmax output layer
Y = Dense(train_Y[0].shape[0], activation='softmax')(Y)

In [17]:
#now build the whole functional model
model = Model(inputs=sub_net_inputs, outputs=Y)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
print('\n')
print("7_Shinoj Mathew")
print("Begin training...")



7_Shinoj Mathew
Begin training...


In [18]:
#train the model
model.fit([train_X] * ensemble_num, train_Y, validation_data=[[test_X] * ensemble_num, test_Y], epochs=epochs, batch_size=batch)

print("Training complete...")
np.set_printoptions(precision=2, suppress=True)
for i in range(len(test_X)):
  print("Prediction: " + str(model.predict([test_X[i].reshape(1, test_X[i].shape[0])] * ensemble_num)) + " | True: " + str(test_Y[i]))


Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 0.8179 - val_loss: 0.9338
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4437 - val_loss: 0.3931
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3244 - val_loss: 0.2185
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1466 - val_loss: 0.1272
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1455 - val_loss: 0.1155
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1274 - val_loss: 0.0978
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0790 - val_loss: 0.0805
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0910 - val_loss: 0.0798
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0669 - val_loss: 0.0797
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0571 - val_loss: 0.1603
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1222 - val_loss: 0.0649
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.